Few Changes To deprecation.py
  1. Adding _decorate_property() to handle
  
     @deprecated()
     
     @property()
     
     def method():
  
  
       
  2. Modifying _decorate_fun() to handle function and

      @property
      
      @deprecated
      
      def method():

In [ ]:
import inspect,warnings
class deprecated(object):

    def __init__(self, extra=''):
        self.extra = extra

    def __call__(self, obj):
        if isinstance(obj, type):
            return self._decorate_class(obj)
        elif inspect.getmembers(obj.__get__,lambda o: isinstance(o,property)) != []:
            return self._decorate_property(obj)                   #for @deprecated \n @property \n funct()
        else:                                                      
            return self._decorate_fun(obj)                        #for method and @property \n @deprecated \n funct()

    def _decorate_property(self,prop):                             
        msg = "Property %s is deprecated; " %prop.fget.__name__
        msg+=self.extra
        def wrapped(*args, **kwargs):
            warnings.warn(msg, category=DeprecationWarning)
            return prop.fget(*args, **kwargs)
        
        wrapped.__name__ = prop.getter.__name__
        wrapped.__doc__ = self._update_doc(prop.__doc__)
        
        return wrapped
    
    def _decorate_class(self, cls):
        msg = "Class %s is deprecated" % cls.__name__
        if self.extra:
            msg += "; %s" % self.extra

        # FIXME: we should probably reset __new__ for full generality
        init = cls.__init__

        def wrapped(*args, **kwargs):
            warnings.warn(msg, category=DeprecationWarning)
            return init(*args, **kwargs)
        cls.__init__ = wrapped

        wrapped.__name__ = '__init__'
        wrapped.__doc__ = self._update_doc(init.__doc__)
        wrapped.deprecated_original = init

        return cls

    def _decorate_fun(self, fun):
        """Decorate function fun"""
         
        

        def wrapped(*args, **kwargs):
            msg = "Function %s is deprecated" % fun.__name__
            if inspect.getargspec(fun)[0][0] == "self":
                Properties_ = inspect.getmembers(args[0].__class__, 
                                      lambda o: isinstance(o, property))
                is_found=False
                for Property_ in Properties_:
                    aa=Property_[1].fget.__name__ ==  fun.__name__ if Property_[1].fget is not None else False
                    bb=Property_[1].fset.__name__ ==  fun.__name__ if Property_[1].fset is not None else False
                    cc=Property_[1].fdel.__name__ ==  fun.__name__ if Property_[1].fdel is not None else False
                   
                    if aa or bb or cc:
                        is_found=True
                        break
                if is_found:   
                    msg = "Property %s is deprecated" % fun.__name__
                
            
            if self.extra:
                msg += "; %s" % self.extra
            
            warnings.warn(msg, category=DeprecationWarning)
            return fun(*args, **kwargs)

        wrapped.__name__ = fun.__name__
        wrapped.__dict__ = fun.__dict__
        wrapped.__doc__ = self._update_doc(fun.__doc__)

        return wrapped

    def _update_doc(self, olddoc):
        newdoc = "DEPRECATED"
        if self.extra:
            newdoc = "%s: %s" % (newdoc, self.extra)
        if olddoc:
            newdoc = "%s\n\n%s" % (newdoc, olddoc)
        return newdoc

Class contaning all such methods and property

In [2]:

class C(object):
    @deprecated("dep")
    def getx(self): 
        print "Output: Deprecated Property fget"
        return self.__x    
    @deprecated("dep")
    def setx(self, value): 
        print "Output: Deprecated Property fset"
        self.__x = value
    @deprecated("dep")    
    def delx(self):  
        print "Output: Deprecated Property fdel"
        del self.__x
        
    @deprecated("dep")
    @property
    def func_1(self):
        return "Output: Deprecated Property"
    
    @property
    @deprecated("dep")
    def Func_2(self):
        return "Output: Deprecated Property"
    
    @deprecated("dep")
    def Func_3(self):
        return "Output: Deprecated Method"
    
    Y = property(fget = getx,fset = setx,fdel = delx, doc = "I'm the 'Y' property.")

In [3]:
a=C()
a.Y=1

Output: Deprecated Property fset


/home/me/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: DeprecationWarning: Property setx is deprecated; dep


In [4]:
a.Y

Output: Deprecated Property fget


/home/me/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: DeprecationWarning: Property getx is deprecated; dep


1

In [5]:
a.func_1()

/home/me/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: DeprecationWarning: Property func_1 is deprecated; dep


'Output: Deprecated Property'

In [6]:
a.Func_2

/home/me/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: DeprecationWarning: Property Func_2 is deprecated; dep


'Output: Deprecated Property'

In [7]:
a.Func_3()

/home/me/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:83: DeprecationWarning: Function Func_3 is deprecated; dep


'Output: Deprecated Method'